In [1]:
# import the training method 
from Server import *

In [ ]:
seed = 923
synthetic_info = dataGenerate(seed = seed, test_samples = 100)
server = Server(logReg(num_features=3, num_classes=2), synthetic_info, train_prn = False, metric = "Demographic disparity", seed = seed)
server.FairBatch(num_rounds = 1, local_epochs = 1, learning_rate = 0.005, optimizer = 'adam', alpha = 1, adaptive_alpha = True)


In [ ]:
seed = 923
synthetic_info = dataGenerate(seed = seed, test_samples = 100)
server = Server(logReg(num_features=3, num_classes=2), synthetic_info, train_prn = False, metric = "Demographic disparity", seed = seed)
server.FairBatch(num_rounds = 1, local_epochs = 1, learning_rate = 0.005, optimizer = 'adam', alpha = 1, adaptive_alpha = True)


In [ ]:
synthetic_info = dataGenerate(seed = 341, test_samples = 100)
server = Server(logReg(num_features=3, num_classes=2), synthetic_info, train_prn = False, metric = "Demographic disparity", seed = 341)
server.FairBatch(num_rounds = 10, local_epochs = 30, learning_rate = 0.005, optimizer = 'adam', alpha = 1, adaptive_alpha = True)


In [ ]:
synthetic_info = dataGenerate(seed = 341, test_samples = 100)
server = Server(logReg(num_features=3, num_classes=2), synthetic_info, train_prn = False, metric = "Demographic disparity", seed = 341)
server.FairBatch(num_rounds = 10, local_epochs = 30, learning_rate = 0.005, optimizer = 'adam', alpha = 1, adaptive_alpha = True)